## SILVER TO GOLD LAYER

### Gold Layer - Trade History


In [1]:
# Import necessary libraries and utility functions
import pandas as pd
from common_utilities import Portfolio, global_path, logger

### Data Processing

- Read and sort trade history data.
- Apply portfolio trade logic.


In [2]:
# Read the CSV file
df_trade_history = pd.read_csv(global_path.tradehistory_silver_file_path)

# Convert 'datetime' to datetime type
df_trade_history["datetime"] = pd.to_datetime(df_trade_history["datetime"])

# Sort the DataFrame by 'datetime'
df_trade_history = df_trade_history.sort_values(by="datetime")

logger.info(
    f"Read SILVER Layer trade history data from: {global_path.tradehistory_silver_file_path}"
)

2024-08-03T00:16:05Z - INFO - Read SILVER Layer trade history data from: C:\Users\prashant.tripathi\Code\Upstox\DATA\SILVER\TradeHistory\TradeHistory_data.csv


### Portfolio Logic Application

- Instantiate Portfolio and apply trade logic.
- Handle expired stocks and round necessary columns.


In [3]:
from pydantic import BaseModel, field_validator
from datetime import datetime
from typing import Optional, Union, Dict, List
from datetime import datetime
from typing import Any, Dict, List, Optional, Union
from pydantic import BaseModel, field_validator


class TradeRecord(BaseModel):
    datetime: datetime
    exchange: Optional[str]
    segment: Optional[str]
    stock_name: str
    side: str
    amount: Union[float, int]
    quantity: Union[float, int]
    price: Union[float, int]
    expiry_date: Optional[datetime]

    @field_validator("expiry_date", mode="before")
    def parse_expiry_date(cls, value):
        try:
            return (
                None
                if str(value) in (None, "nan", "")
                else datetime.strptime(str(value), "%Y-%m-%d")
            )
        except ValueError as e:
            raise ValueError(f"Invalid expiry date format: {e}")

In [4]:
class Stock(BaseModel):
    stock_name: str
    expiry_date: Optional[datetime]

    holding_quantity: float = 0.0
    avg_price: float = 0.0
    holding_amount: float = 0.0
    trade_history: List[Dict[str, Any]] = []

    def check_expired(self):
        # Check if the stock is expired and reset holdings if necessary
        if self.expiry_date and datetime.now() > self.expiry_date:
            pass

    def trade(self, trade_record: TradeRecord):
        # BUY: positive position, SELL: negative position
        trade_record.quantity = (
            trade_record.quantity
            if trade_record.side == "BUY"
            else (-1) * trade_record.quantity
        )

        if (self.holding_quantity * trade_record.quantity) >= 0:
            # Realized PnL
            pnl_amount = 0
            pnl_percentage = 0
            # Avg open price
            self.avg_price = (
                (self.avg_price * self.holding_quantity)
                + (trade_record.price * trade_record.quantity)
            ) / (self.holding_quantity + trade_record.quantity)
        else:
            # Calculate PnL and percentage
            pnl_amount = (
                (trade_record.price - self.avg_price)
                * min(abs(trade_record.quantity), abs(self.holding_quantity))
                * (abs(self.holding_quantity) / self.holding_quantity)
            )
            pnl_percentage = (
                pnl_amount
                / (
                    self.avg_price
                    * min(
                        abs(trade_record.quantity), abs(self.holding_quantity)
                    )
                )
            ) * 100

            # Check if it is close-and-open
            if abs(trade_record.quantity) > abs(self.holding_quantity):
                self.avg_price = trade_record.price

        # Net position
        self.holding_quantity += trade_record.quantity
        self.holding_amount = self.holding_quantity * self.avg_price

        trade_record.quantity = abs(trade_record.quantity)

        trade_data = trade_record.model_dump()
        trade_data.update(self.model_dump(exclude=self.trade_history))
        trade_data.update({
            "pnl_amount":pnl_amount,
            "pnl_percentage":pnl_percentage,
                           })
        
        self.trade_history.append(trade_data)

    def is_expired(self) -> bool:
        """
        Checks if a expiry_date is in the past.

        Returns:
            bool: True if the date is in the past, False otherwise.
        """

        return (
            datetime.today() > self.expiry_date
            if self.expiry_date is not None
            else False
        )

In [5]:
class Portfolio:
    def __init__(self):
        self.stocks: Dict[str, Stock] = {}

    def trade(self, trade_record: TradeRecord):
        print(trade_record)
        if trade_record.stock_name not in self.stocks:
            self.stocks[trade_record.stock_name] = Stock(
                stock_name=trade_record.stock_name,
                expiry_date=trade_record.expiry_date,
            )
        self.stocks[trade_record.stock_name].trade(trade_record)

    def check_expired_stocks(self):
        for stock in self.stocks.values():
            stock.check_expired()

    def get_trade_history(self):
        trade_history = []
        for stock in self.stocks.values():
            trade_history += stock.trade_history
        return trade_history

In [6]:
# Apply the trade logic to each row of the DataFrame
# Instantiate the Portfolio object
portfolio = Portfolio()

for record in (
    df_trade_history[df_trade_history["stock_name"] == "TATAPOWER"]
    .astype(str)
    .to_dict(orient="records")
):
    portfolio.trade(TradeRecord(**record))

portfolio.check_expired_stocks()

# Create a DataFrame from the processed data
df_trade_history = pd.DataFrame(data=portfolio.get_trade_history())

df_trade_history

ValidationError: 2 validation errors for TradeRecord
pnl_amount
  Field required [type=missing, input_value={'datetime': '2021-02-08 ...', 'expiry_date': 'nan'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.1.2/v/missing
pnl_percentage
  Field required [type=missing, input_value={'datetime': '2021-02-08 ...', 'expiry_date': 'nan'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.1.2/v/missing

### Final Processing and Export

- Select and sort relevant columns.
- Save the processed data as a CSV file in the Gold layer.


In [ ]:
portfolio.get_trade_history()

In [ ]:
# Round the values in the columns to two decimal places
# df_trade_history = df_trade_history.round(2)

# # Data cleanup
# df_trade_history = df_trade_history.replace("nan", None)
# df_trade_history = df_trade_history.fillna(None)

# # Save the result as a CSV file



# # Sort the DataFrame by 'segment', 'stock_name', and 'datetime'



# df_trade_history = df_trade_history.sort_values(



#     by=["segment", "stock_name", "datetime"]



# )



# # Select relevant columns



# df_trade_history = df_trade_history[



#     [
#         "datetime",



#         "exchange",
#         "segment",



#         "stock_name",
#         "scrip_code",



#         "expiry_date",
#         "side",



#         "quantity",
#         "price",
#         "amount",



#         "holding_quantity",



#         "avg_price",
#         "holding_amount",
#         "pnl_amount",
#         "pnl_percentage",



#     ]



# ]



# df_trade_history.to_csv(global_path.tradehistory_gold_file_path, index=None)



# logger.info("GOLD Layer CSV file for trade history successfully created at:")



# logger.info(global_path.tradehistory_gold_file_path.resolve())



# # Log the DataFrame info



# df_trade_history.info()